In [51]:
'''reading the sudoku problem from file'''

arq = open('sudoku_2', 'r')
sudoku = arq.readlines()
grid = []

count_plain = 0
count_fv = 0
count_mvs = 0

N = 0
for i in range(len(sudoku)):
    
    if(i == 0):
        N = sudoku[i]
    else:
        grid.append(sudoku[i].split())

arq.close()


def show_grid():
    for i in range(len(grid)):          
        print(grid[i])
    print("\n")

In [39]:
'''generate the list with the blank spaces of the grid '''

def empty_list(grid):
    
    empty = {}
    k = 0
    for i in range(len(grid)):
        for j in range(len(grid)):
            if(grid[i][j] == "0"):
                empty[k] = [i,j]
                k = k + 1
            
    return empty
    

In [40]:
'''verification of lin, col, subgrids'''

def check_lin(grid, i, chosen):
    
    if(chosen in grid[i]):
        return False
    else:
        return True
    
def check_col(grid, j, chosen):
    
    for i in range(len(grid)):
        if(chosen == grid[i][j]):
            return False
    
    return True

def check_subgrid(grid, i, j, chosen):
    
    subl = i - i%3
    subc = j - j%3
    aux = subc    
        
    for subl in range(subl, subl+3):
        for subc in range(subc, subc+3):
            if(grid[subl][subc] == chosen):
                return False
         
        subc = aux
        
    return True
    
    

In [41]:
''' backtracking algorithm '''

def backtrack(grid):
    
    global count_plain
    
    empty = empty_list(grid)
    
    if (empty == {}):
        return True
            
    spoti, spotj = empty[0][0], empty[0][1]

    for i in range(1,10):
        
        count_plain = count_plain + 1
        
        chosen = str(i)
        
        if(check_lin(grid, spoti, chosen) == True and check_col(grid, spotj, chosen) == True and   
           check_subgrid(grid, spoti, spotj, chosen) == True):

            grid[spoti][spotj] = chosen

            result = backtrack(grid)

            if(result == False):
                grid[spoti][spotj] = "0"
            else:
                return True
        

    return False    
    

In [42]:
'''foward verification functions'''

def ver_values_lin(grid, i, pvalues):
    
    newpvalues = []
    
    for j in range(len(pvalues)):
        if(pvalues[j] not in grid[i]): 
            newpvalues.append(pvalues[j])
            
                
    newpvalues = [x for x in pvalues if x in newpvalues] 
    
    return newpvalues

def ver_values_col(grid, j, pvalues):
    
    col_values = []
    newpvalues = []
    
    for k in range(len(grid)):
        col_values.append(grid[k][j])
        
    for i in range(len(pvalues)):
        if(pvalues[i] not in col_values):
            newpvalues.append(pvalues[i])
    
    newpvalues = [x for x in pvalues if x in newpvalues]
        
    return newpvalues

def ver_values_subgrid(grid, lin , col, pvalues):
    
    subl = lin - lin%3
    subc = col - col%3
    aux = subc  
    
    newpvalues = []
    subgrid_values = []
    
    for subl in range(subl, subl+3):
        for subc in range(subc, subc+3):
            subgrid_values.append(grid[subl][subc])
        subc = aux
                    
    for i in range(len(pvalues)):
        if(pvalues[i] not in subgrid_values):
            newpvalues.append(pvalues[i])
        
    newpvalues = [x for x in pvalues if x in newpvalues]
    
    return newpvalues
        

def foward_verification(grid, spoti, spotj):
    
    tvalues = {}
          
        
    pvalues = ["1","2","3","4","5","6","7","8","9"]
        
    pvalues = ver_values_lin(grid, spoti, pvalues)
    pvalues = ver_values_col(grid , spotj, pvalues)
    pvalues = ver_values_subgrid(grid, spoti , spotj, pvalues)
            
    return pvalues

In [47]:
''' backtracking algorithmc with foward verification '''

def backtrack_fv(grid):
    
    global count_fv
    
    empty = empty_list(grid)

    if (empty == {}):
        return True
            
    spoti, spotj = empty[0][0], empty[0][1]
    
    pvalues = foward_verification(grid, spoti, spotj)
    
    if(pvalues == []):
        return False
    
    for i in range(0,len(pvalues)):
        
        count_fv = count_fv + 1
                
        chosen = str(pvalues[i])
        
        if(check_lin(grid, spoti, chosen) == True and check_col(grid, spotj, chosen) == True and   
           check_subgrid(grid, spoti, spotj, chosen) == True):

            grid[spoti][spotj] = chosen

            result = backtrack_fv(grid)

            if(result == False):
                grid[spoti][spotj] = "0"
            else:
                return True
            
    return False

In [44]:
''' mvs algorithm '''

def mvs(grid, empty):
    
    spoti = 0
    spotj = 0
    
    min = foward_verification(grid, empty[0][0], empty[0][1])
    spoti = empty[0][0]
    spotj = empty[0][1]
    
    for i in range(1, len(empty)):
        aux = foward_verification(grid, empty[i][0], empty[i][1])
        if(len(min) > len(aux)):
            min = foward_verification(grid, empty[i][0], empty[i][1])
            spoti = empty[i][0]
            spotj = empty[i][1]
            
    return min, spoti, spotj
    

In [45]:
''' backtracking algorithmc with foward verification and mvs '''

def backtrack_fv_mvs(grid):
    
    global count_mvs
    
    empty = empty_list(grid)

    if (empty == {}):
        return True
                
    pvalues, spoti, spotj = mvs(grid, empty)
    
    if(pvalues == []):
        return False
    
    for i in range(0,len(pvalues)):
        
        count_mvs = count_mvs + 1
                
        chosen = str(pvalues[i])
        
        if(check_lin(grid, spoti, chosen) == True and check_col(grid, spotj, chosen) == True and   
           check_subgrid(grid, spoti, spotj, chosen) == True):

            grid[spoti][spotj] = chosen

            result = backtrack_fv_mvs(grid)

            if(result == False):
                grid[spoti][spotj] = "0"
            else:
                return True
            
    return False

In [36]:
show_grid()
backtrack(grid)
print(count_plain)
show_grid()

['0', '6', '0', '1', '0', '4', '0', '5', '0']
['0', '0', '8', '3', '0', '5', '6', '0', '0']
['2', '0', '0', '0', '0', '0', '0', '0', '1']
['8', '0', '0', '4', '0', '7', '0', '0', '6']
['0', '0', '6', '0', '0', '0', '3', '0', '0']
['7', '0', '0', '9', '0', '1', '0', '0', '4']
['5', '0', '0', '0', '0', '0', '0', '0', '2']
['0', '0', '7', '2', '0', '6', '9', '0', '0']
['0', '4', '0', '5', '0', '8', '0', '7', '0']


14096
['9', '6', '3', '1', '7', '4', '2', '5', '8']
['1', '7', '8', '3', '2', '5', '6', '4', '9']
['2', '5', '4', '6', '8', '9', '7', '3', '1']
['8', '2', '1', '4', '3', '7', '5', '9', '6']
['4', '9', '6', '8', '5', '2', '3', '1', '7']
['7', '3', '5', '9', '6', '1', '8', '2', '4']
['5', '8', '9', '7', '1', '3', '4', '6', '2']
['3', '1', '7', '2', '4', '6', '9', '8', '5']
['6', '4', '2', '5', '9', '8', '1', '7', '3']




In [50]:
show_grid()
backtrack_fv(grid)
print(count_fv)
show_grid()



['0', '6', '0', '1', '0', '4', '0', '5', '0']
['0', '0', '8', '3', '0', '5', '6', '0', '0']
['2', '0', '0', '0', '0', '0', '0', '0', '1']
['8', '0', '0', '4', '0', '7', '0', '0', '6']
['0', '0', '6', '0', '0', '0', '3', '0', '0']
['7', '0', '0', '9', '0', '1', '0', '0', '4']
['5', '0', '0', '0', '0', '0', '0', '0', '2']
['0', '0', '7', '2', '0', '6', '9', '0', '0']
['0', '4', '0', '5', '0', '8', '0', '7', '0']


1589
['9', '6', '3', '1', '7', '4', '2', '5', '8']
['1', '7', '8', '3', '2', '5', '6', '4', '9']
['2', '5', '4', '6', '8', '9', '7', '3', '1']
['8', '2', '1', '4', '3', '7', '5', '9', '6']
['4', '9', '6', '8', '5', '2', '3', '1', '7']
['7', '3', '5', '9', '6', '1', '8', '2', '4']
['5', '8', '9', '7', '1', '3', '4', '6', '2']
['3', '1', '7', '2', '4', '6', '9', '8', '5']
['6', '4', '2', '5', '9', '8', '1', '7', '3']




In [52]:
show_grid()
backtrack_fv_mvs(grid)
print(count_mvs)
show_grid()

['0', '6', '0', '1', '0', '4', '0', '5', '0']
['0', '0', '8', '3', '0', '5', '6', '0', '0']
['2', '0', '0', '0', '0', '0', '0', '0', '1']
['8', '0', '0', '4', '0', '7', '0', '0', '6']
['0', '0', '6', '0', '0', '0', '3', '0', '0']
['7', '0', '0', '9', '0', '1', '0', '0', '4']
['5', '0', '0', '0', '0', '0', '0', '0', '2']
['0', '0', '7', '2', '0', '6', '9', '0', '0']
['0', '4', '0', '5', '0', '8', '0', '7', '0']


54
['9', '6', '3', '1', '7', '4', '2', '5', '8']
['1', '7', '8', '3', '2', '5', '6', '4', '9']
['2', '5', '4', '6', '8', '9', '7', '3', '1']
['8', '2', '1', '4', '3', '7', '5', '9', '6']
['4', '9', '6', '8', '5', '2', '3', '1', '7']
['7', '3', '5', '9', '6', '1', '8', '2', '4']
['5', '8', '9', '7', '1', '3', '4', '6', '2']
['3', '1', '7', '2', '4', '6', '9', '8', '5']
['6', '4', '2', '5', '9', '8', '1', '7', '3']


